# Aiken Hello World (CLI version)

In this tutorial we'll be running and end-to-end example of how to build and execute a Cardano validator (aka: smart contract) using only Aiken and the Cardano node cli.

This jupyter notebook will guide you through the process of:

- making some changes to our Aiken code
- compiling our Aiken code into Plutus core
- locking some ADA from our wallet into the script address
- un-locking that ADA back into our wallet

## Make some changes to our Aiken code

The source code for our validator script is located in `validators/hello_world.ak`. If you haven't read the official [Aiken introduction](https://aiken-lang.org), this would be a good time to do so.

We won't go into much detail, but is important to understand the following about our particular "hello world" validator:

- when we lock assets, the datum of the input specifies an "owner" key that will be required to unlock the assets
- when we unlock assets, we need to pass a redeemer that holds a specific hardcoded phrase

The default value used in this repo is `Hello, World!`.

✅ task: edit the Aiken source code and change the hardcoded phrase to anything you want.

## Turn our Aiken code into UPLC

Cardano understands UPLC, so we'll be using the `aiken` CLI to turn our code into the required bytestring.

To make sure we haven't messed up anything on our source code, we'll use the `aiken check` command to validate the syntax and run required tests (if any).

In [255]:
aiken check

    Compiling aiken-lang/stdlib main (/config/workspace/repo/build/packages/aiken-lang-stdlib)
    Compiling aiken-lang/hello_world 0.0.0 (/config/workspace/repo)

Summary
    0 errors, 0 warnings


If the check command succeded, we can now run the `aiken build` command to get our UPLC. The output will be wrapped in a blueprint (CIP-57) json file named `plutus.json`.

In [256]:
aiken build

jq . plutus.json

    Compiling aiken-lang/stdlib main (/config/workspace/repo/build/packages/aiken-lang-stdlib)
    Compiling aiken-lang/hello_world 0.0.0 (/config/workspace/repo)
   Generating project's blueprint (/config/workspace/repo/plutus.json)

Summary
    0 errors, 0 warnings
{
  "preamble": {
    "title": "aiken-lang/hello_world",
    "description": "Aiken contracts for project 'aiken-lang/hello_world'",
    "version": "0.0.0",
    "plutusVersion": "v2",
    "license": "Apache-2.0"
  },
  "validators": [
    {
      "title": "hello_world.hello_world",
      "datum": {
        "title": "datum",
        "schema": {
          "$ref": "#/definitions/hello_world~1Datum"
        }
      },
      "redeemer": {
        "title": "redeemer",
        "schema": {
          "$ref": "#/definitions/hello_world~1Redeemer"
        }
      },
      "compiledCode": "5901fe0100003232323232323232323232322223232533300a3232533300c002100114a06464660026eb0cc024c02ccc024c02c019200048040dd7198049805802240006002002444a66

Since we'll be using the `cardano-cli` to submit our transaction, we need to run an extra step to convert our blueprint file into the json format that the cardano cli expects. For this, we have a very handy command in the Aiken cli just for that:

In [257]:
aiken blueprint convert > script.json

jq . script.json

{
  "type": "PlutusScriptV2",
  "description": "Generated by Aiken",
  "cborHex": "5902015901fe0100003232323232323232323232322223232533300a3232533300c002100114a06464660026eb0cc024c02ccc024c02c019200048040dd7198049805802240006002002444a66602600429404c8c94ccc040cdc78010018a5113330050050010033016003375c602800466e3cdd71980318040012400091011541696b656e20434c492068656c6c6f20776f726c6400149858cc028c94ccc028cdc3a400000226464a66602260260042930a99807249334c6973742f5475706c652f436f6e73747220636f6e7461696e73206d6f7265206974656d73207468616e2065787065637465640016375c6022002601000a2a660189212b436f6e73747220696e64657820646964206e6f74206d6174636820616e7920747970652076617269616e7400163008004004330093253330093370e900000089919299980818090010a4c2a6601a9201334c6973742f5475706c652f436f6e73747220636f6e7461696e73206d6f7265206974656d73207468616e2065787065637465640016375c6020002600e0062a660169212b436f6e73747220696e64657820646964206e6f74206d6174636820616e7920747970652076617269616e740016300700200233001001480008888

As a last step, we'll use `aiken` CLI to generate the address for our script and store it in the `SCRIPT_ADDRESS` variable:

In [258]:
SCRIPT_ADDRESS=$(aiken address)
echo "our script is: $SCRIPT_ADDRESS"


Summary
    0 errors, 0 warnings
our script is: addr_test1wqggwwu5thmrwfew76t2nvxr2jtlve026pdhmawh4563jec5dlvav


## Lock some tADA in our script

What follows has little to do with Aiken language. What we want to do is send some tADA from a wallet that we own to the address of the script.

Lets use the `cardano-cli` to create our key pair. We'll make sure that the file doesn't already exist to avoid overwriting our secret key.

Once we have our key pair, we'll store the address and keyhash of the wallet in a variable so that we can use it in later operations.

In [259]:
if [ ! -e "wallet.sk" ]; then
    cardano-cli address key-gen --verification-key-file wallet.vk --signing-key-file wallet.sk
fi

WALLET_ADDRESS=$(cardano-cli address build \
    --payment-verification-key-file ./wallet.vk \
    --testnet-magic $CARDANO_NODE_MAGIC)

echo "my wallet address: $WALLET_ADDRESS"

WALLET_KEYHASH=$(cardano-cli address key-hash \
    --payment-verification-key-file wallet.vk)

echo "my wallet keyhash: $WALLET_KEYHASH"

my wallet address: addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn
my wallet keyhash: 024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f


Our brand new wallet is empty, we need to get some tADA before we can do anything interesting with it. Go to the [testnet faucet](https://docs.cardano.org/cardano-testnet/tools/faucet) and request some for your address (you can copy the value from the output in the previous cell).

If everything goes well, after a few seconds you should receive some assets in your wallet. Run the following snippet to check the state of your wallet. The output holds a reference to the available UTxOs owned by your address. the output will be save as `wallet.state` for later reference. 

In [260]:
cardano-cli query utxo \
    --address $WALLET_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file wallet.state

jq . wallet.state

{
  "935ecca1bcf274e4019b5cb9378ce22fb2146276da73183daa1661672456b050#1": {
    "address": "addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 9989832475
    }
  }
}


Next, we'll select some UTxO from our wallet to be used as input in the transaction that will lock tADA in the script address. For that, the following snippet select the 1st UTxO from the `wallet.state` file previously generated and saved the utxo ref as a variable named `INPUT_UTXO`.

💡 tip: If you want to select some other UTxO other than the 1st one, change the `keys[0]` part of the snippet to whatever index you want.

In [261]:
INPUT_UTXO=$(jq -r 'keys[0]' wallet.state)

echo $INPUT_UTXO

935ecca1bcf274e4019b5cb9378ce22fb2146276da73183daa1661672456b050#1


Before we submit the transaction, we need to prepare our datum. Our particular hello-world script expects a datum that holds a reference to the pub keyhash that represents the owner of the UTxO. We'll use the previously assigned `WALLET_KEYHASH` variable to create a `datum.json` file with the format that `cardano-cli` expects for a datum.

In [262]:
echo "{\"constructor\" : 0, \"fields\": [{ \"bytes\": \"$WALLET_KEYHASH\" }]}" > datum.json

jq . datum.json

{
  "constructor": 0,
  "fields": [
    {
      "bytes": "024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f"
    }
  ]
}


We should now be ready to build our transaction. The following snippet uses the `cardano-cli` to generate a `lock.tx` json file that contains the CBOR for our (un-signed) transaction.

Notice the following:
- the input comes from the `INPUT_UTXO` variable, which is the UTxO ref that we picked from our wallet
- the output sends 10 tADA to the address of our script stored in the `SCRIPT_ADDRESS` variable
- we include an inline-datum value coming from the `datum.json` file previously generated

In [263]:
cardano-cli transaction build \
    --babbage-era \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-in $INPUT_UTXO \
    --tx-out $SCRIPT_ADDRESS+10000000 \
    --tx-out-inline-datum-file datum.json \
    --change-address $WALLET_ADDRESS \
    --out-file lock.tx

cardano-cli transaction view --tx-file lock.tx

Estimated transaction fee: Lovelace 167525
auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 167525 Lovelace
inputs:
- 935ecca1bcf274e4019b5cb9378ce22fb2146276da73183daa1661672456b050#1
metadata: null
mint: null
outputs:
- address: addr_test1wqggwwu5thmrwfew76t2nvxr2jtlve026pdhmawh4563jec5dlvav
  address era: Shelley
  amount:
    lovelace: 10000000
  datum:
    constructor: 0
    fields:
    - bytes: 024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f
  network: Testnet
  payment credential script hash: 10873b945df637272ef696a9b0c35497f665ead05b7df5d7ad351967
  reference script: null
  stake reference: null
- address: addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn
  address era: Shelley
  amount:
    lovelace: 9979664950
  datum: null
  network: Testnet
  payment credential key hash: 024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f
  reference script: null
  stake reference: null
reference inputs: []
required signers (paym

If everything went well, we should be able to sing and submit the transaction using the following snippet:

In [264]:
cardano-cli transaction sign \
    --tx-file lock.tx \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --signing-key-file wallet.sk \
    --out-file lock.signed

cardano-cli transaction submit --testnet-magic $CARDANO_NODE_MAGIC --tx-file lock.signed

Transaction successfully submitted.


After a few seconds, our script address should hold a new UTxO with 10 tADA and an inline datum. The following snippets uses `cardano-cli` to check the state of the script and store the output in the `script.state` file.

In [273]:
cardano-cli query utxo \
    --address $SCRIPT_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file script.state

jq . script.state

{
  "9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#0": {
    "address": "addr_test1wqggwwu5thmrwfew76t2nvxr2jtlve026pdhmawh4563jec5dlvav",
    "datum": null,
    "inlineDatum": {
      "constructor": 0,
      "fields": [
        {
          "bytes": "024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f"
        }
      ]
    },
    "inlineDatumhash": "efe221dfe058d7917f052444f34ada37ee32cc3b739acd3bff47d34671527e21",
    "referenceScript": null,
    "value": {
      "lovelace": 10000000
    }
  }
}


If we didn't mess up anything, our tADA should be safely locked by our script. Now comes the important part, unlocking our assets! 😬

## Unlock our tADA and send it back to our wallet

Locking assets in a script is easy, no custom code is actually executed. The hard part comes during un-locking of the assets, which is when our Aiken code (UPLC at this stage) will decide if the UTxO can be moved out from the script address or not.

1st step is to select which UTxO locked at the script address we want to retrieve. We'll do something similar to when we selected the UTxO for locking. The following snippet picks the first UTxO of the set and store the ref in the `TO_UNLOCK_UTXO` variable.

💡 tip: If you want to select some other UTxO other than the 1st one, change the `keys[0]` part of the snippet to whatever index you want.

In [274]:
TO_UNLOCK_UTXO=$(jq -r 'keys[0]' script.state)

echo $TO_UNLOCK_UTXO

9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#0


Next, we'll prepare our redeemer. If you remember the logic in our Aiken code, the validator will only pass if the redeemer contains the same value that was hardcoded as part of the script. The following snippet turns the value of the `MSG` variable into it's hex representation and stores the whole redeemer in the `redeemer.json` file.

✅ task: set the `MSG` variable to whatever message you used in the `validators/hello_world.ak` file.

In [275]:
MSG="Aiken CLI hello world"

BYTES=$(echo -n $MSG | xxd -p )
echo "{\"constructor\" : 0, \"fields\": [{ \"bytes\": \"$BYTES\" }]}" > redeemer.json

jq . redeemer.json

{
  "constructor": 0,
  "fields": [
    {
      "bytes": "41696b656e20434c492068656c6c6f20776f726c64"
    }
  ]
}


💡 tip: notice that we use the `-n` for the echo command to avoid the line break at the end. I spent 2hs debugging why my tx was failing.

Next, since this transaction will execute our custom validator, we need to define some collateral. The following snippet refreshes the UTxO state of our wallet so that we can pick which one to use for collateral.

In [276]:
cardano-cli query utxo \
    --address $WALLET_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file wallet.state

jq . wallet.state

{
  "9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#1": {
    "address": "addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 9979664950
    }
  }
}


We pick the 1st UTxO in the set as collateral and store it in a variable named `COLLATERAL_UTXO`.

💡 tip: If you want to select some other UTxO other than the 1st one, change the `keys[0]` part of the snippet to whatever index you want.

In [277]:
COLLATERAL_UTXO=$(jq -r 'keys[0]' wallet.state)

echo "our collateral UTxO is: $COLLATERAL_UTXO"

our collateral UTxO is: 9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#1


Next step, we need to query the protocol parameters. Since our transaction constains a validator, the protocol uses the parameters to define the fee that will be required for the execution. The following snippet uses `cardano-cli` to query the params from the node and store them in the `params.json` file.

In [278]:
cardano-cli query protocol-parameters --testnet-magic $CARDANO_NODE_MAGIC > params.json

jq . params.json

{
  "collateralPercentage": 150,
  "costModels": {
    "PlutusScriptV1": {
      "addInteger-cpu-arguments-intercept": 205665,
      "addInteger-cpu-arguments-slope": 812,
      "addInteger-memory-arguments-intercept": 1,
      "addInteger-memory-arguments-slope": 1,
      "appendByteString-cpu-arguments-intercept": 1000,
      "appendByteString-cpu-arguments-slope": 571,
      "appendByteString-memory-arguments-intercept": 0,
      "appendByteString-memory-arguments-slope": 1,
      "appendString-cpu-arguments-intercept": 1000,
      "appendString-cpu-arguments-slope": 24177,
      "appendString-memory-arguments-intercept": 4,
      "appendString-memory-arguments-slope": 1,
      "bData-cpu-arguments": 1000,
      "bData-memory-arguments": 32,
      "blake2b_256-cpu-arguments-intercept": 117366,
      "blake2b_256-cpu-arguments-slope": 10475,
      "blake2b_256-memory-arguments": 4,
      "cekApplyCost-exBudgetCPU": 23000,
      "cekApplyCost-exBudgetMemory": 100,
      "cekBuiltinCos

We're now ready to build our unlock transaction. The following snippet uses the `cardano-cli` to preapre the transaction CBOR and store the result in the `unlock.tx` file.

Notice the following:
- our input is the locked UTxO as specified in the `TO_UNLOCK_UTXO` variable
- we include a redeemer which is defined in the `redeemer.json` file
- we indicate that the datum should be avialbe as part of the input UTxO
- we specify the code for our custom validator by referencing the `script.json` file
- we specify our wallet address as the destinatary
- we need to explicitely specify a required signer so that our wallet key is available to the script

In [279]:
cardano-cli transaction build \
    --babbage-era \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --required-signer wallet.sk \
    --protocol-params-file params.json \
    --tx-in $TO_UNLOCK_UTXO \
    --tx-in-redeemer-file redeemer.json \
    --tx-in-inline-datum-present \
    --tx-in-script-file script.json \
    --tx-in-collateral $COLLATERAL_UTXO \
    --change-address $WALLET_ADDRESS \
    --out-file unlock.tx

cardano-cli transaction view --tx-file unlock.tx

Estimated transaction fee: Lovelace 205022
auxiliary scripts: null
certificates: null
collateral inputs:
- 9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#1
era: Babbage
fee: 205022 Lovelace
inputs:
- 9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#0
metadata: null
mint: null
outputs:
- address: addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn
  address era: Shelley
  amount:
    lovelace: 9794978
  datum: null
  network: Testnet
  payment credential key hash: 024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f
  reference script: null
  stake reference: null
reference inputs: []
required signers (payment key hashes needed for scripts):
- 024582de82855cd6da2ed3ff3889802ee3be190905d871270da7022f
return collateral:
  address: addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn
  address era: Shelley
  amount:
    lovelace: 9979357417
  datum: null
  network: Testnet
  payment credential key hash: 024582de82855cd6da2ed3ff38

We can now sign and submit the the transaction.

In [280]:
cardano-cli transaction sign \
    --tx-file unlock.tx \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --signing-key-file wallet.sk \
    --out-file unlock.signed

cardano-cli transaction submit --testnet-magic $CARDANO_NODE_MAGIC --tx-file unlock.signed

Transaction successfully submitted.


If everything went well, we should see our unlocked tADA safely returned to our wallet.

In [283]:
cardano-cli query utxo \
    --address $WALLET_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file wallet.state

jq . wallet.state

{
  "9069eb6aad080873e1a7bba8606bce85c7cda512eae492138b8e3dbc805f0b1a#1": {
    "address": "addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 9979664950
    }
  },
  "d8acf216f1532c8444eede3460cda8d998bc391b694e7db8f37011513d154299#0": {
    "address": "addr_test1vqpytqk7s2z4e4k69mfl7wyfsqhw80sepyzasuf8pknsytc2keepn",
    "datum": null,
    "datumhash": null,
    "inlineDatum": null,
    "referenceScript": null,
    "value": {
      "lovelace": 9794978
    }
  }
}


And the script address should be missing the unlocked UTxO.

In [284]:
cardano-cli query utxo \
    --address $SCRIPT_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --out-file script.state

jq . script.state

{}
